## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Sao Filipe,CV,2022-08-11 04:28:00,14.8961,-24.4956,77.23,88,68,8.86,broken clouds
1,1,Atuona,PF,2022-08-11 04:28:01,-9.8000,-139.0333,76.95,77,2,20.13,clear sky
2,2,Padang,ID,2022-08-11 04:27:23,-0.9492,100.3543,86.05,64,85,4.83,overcast clouds
3,3,Tuatapere,NZ,2022-08-11 04:28:01,-46.1333,167.6833,46.04,62,21,4.25,few clouds
4,4,Isangel,VU,2022-08-11 04:23:38,-19.5500,169.2667,77.11,57,75,11.50,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Sao Filipe,CV,2022-08-11 04:28:00,14.8961,-24.4956,77.23,88,68,8.86,broken clouds
1,1,Atuona,PF,2022-08-11 04:28:01,-9.8000,-139.0333,76.95,77,2,20.13,clear sky
2,2,Padang,ID,2022-08-11 04:27:23,-0.9492,100.3543,86.05,64,85,4.83,overcast clouds
4,4,Isangel,VU,2022-08-11 04:23:38,-19.5500,169.2667,77.11,57,75,11.50,broken clouds
6,6,Atar,MR,2022-08-11 04:28:02,20.5169,-13.0499,89.47,16,100,10.42,overcast clouds
7,7,Marsh Harbour,BS,2022-08-11 04:28:02,26.5412,-77.0636,82.27,74,1,11.95,clear sky
10,10,Kavieng,PG,2022-08-11 04:23:35,-2.5744,150.7967,83.16,72,72,7.92,broken clouds
12,12,Pervomayskoye,RU,2022-08-11 04:28:04,43.9190,46.7102,83.66,47,0,21.12,clear sky
18,18,Winnemucca,US,2022-08-11 04:28:06,40.9730,-117.7357,75.25,64,20,12.66,few clouds
20,20,Kropotkin,RU,2022-08-11 04:22:50,45.4375,40.5756,75.36,50,17,12.28,few clouds


In [11]:
# 4a. Determine if there are any empty rows.
# count cells with missing values
print(sum(preferred_cities_df.isnull().values.ravel()))

# count the rows in each column
preferred_cities_df.count()

0


City_ID        184
City           184
Country        184
Date           184
Lat            184
Lng            184
Max Temp       184
Humidity       184
Cloudiness     184
Wind Speed     184
Description    184
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID        184
City           184
Country        184
Date           184
Lat            184
Lng            184
Max Temp       184
Humidity       184
Cloudiness     184
Wind Speed     184
Description    184
dtype: int64

In [27]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Sao Filipe,CV,77.23,broken clouds,14.8961,-24.4956,
1,Atuona,PF,76.95,clear sky,-9.8000,-139.0333,
2,Padang,ID,86.05,overcast clouds,-0.9492,100.3543,
4,Isangel,VU,77.11,broken clouds,-19.5500,169.2667,
6,Atar,MR,89.47,overcast clouds,20.5169,-13.0499,
7,Marsh Harbour,BS,82.27,clear sky,26.5412,-77.0636,
10,Kavieng,PG,83.16,broken clouds,-2.5744,150.7967,
12,Pervomayskoye,RU,83.66,clear sky,43.9190,46.7102,
18,Winnemucca,US,75.25,few clouds,40.9730,-117.7357,
20,Kropotkin,RU,75.36,few clouds,45.4375,40.5756,


In [28]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json() 
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [40]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np

clean_hotel_df = hotel_df.replace("", np.nan)
clean_hotel_df = clean_hotel_df.dropna()

clean_hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Sao Filipe,CV,77.23,broken clouds,14.8961,-24.4956,Hotel Xaguate
1,Atuona,PF,76.95,clear sky,-9.8000,-139.0333,Hôtel Hiva Oa Hanakee Lodge
2,Padang,ID,86.05,overcast clouds,-0.9492,100.3543,Mercure Padang
4,Isangel,VU,77.11,broken clouds,-19.5500,169.2667,Tanna Lodge
6,Atar,MR,89.47,overcast clouds,20.5169,-13.0499,Oumou Elghoura
7,Marsh Harbour,BS,82.27,clear sky,26.5412,-77.0636,Abaco Beach Resort
10,Kavieng,PG,83.16,broken clouds,-2.5744,150.7967,Nusa Island Retreat
18,Winnemucca,US,75.25,few clouds,40.9730,-117.7357,Winners Inn Casino
20,Kropotkin,RU,75.36,few clouds,45.4375,40.5756,Hotel Europe Kropotkin
22,Kapaa,US,84.18,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort


In [41]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [48]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
# I also think Hotel Name is important, so I'm adding that! 

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [49]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))